# Logistic Regression (Ridge)

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [4]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=RandomForestClassifier(random_state=42), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=6)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7feefd707cd0>>)
Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f90485c8cd0>>)
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f3d73a36cd0>>)
Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f3d69eaf040>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer=

In [5]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/random_forest.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.850644,0.851396,0.853803,0.850959,0.918879,CountVectorizer(tokenizer=<bound method TweetT...
1,0.832374,0.834380,0.840930,0.833312,0.911415,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.809579,0.810346,0.813251,0.810183,0.891217,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.773216,0.775991,0.785775,0.775767,0.863120,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.779996,0.781983,0.788564,0.781593,0.867920,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.846650,0.847374,0.849606,0.846935,0.916304,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.829822,0.831432,0.836297,0.830436,0.908750,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.809160,0.810010,0.813251,0.809809,0.892261,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.773682,0.776847,0.787983,0.776678,0.863382,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.776250,0.778805,0.788110,0.778655,0.869366,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [6]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.850644
recall_macro                                                0.851396
precision_macro                                             0.853803
accuracy                                                    0.850959
auc_score                                                   0.918879
vectorizer         CountVectorizer(tokenizer=<bound method TweetT...
Name: 0, dtype: object

Vectorizer: CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f90485c8cd0>>)


In [7]:
pd.read_csv("../../results/random_forest.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.850644,0.851396,0.853803,0.850959,0.918879,CountVectorizer(tokenizer=<bound method TweetT...
1,0.832374,0.834380,0.840930,0.833312,0.911415,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.809579,0.810346,0.813251,0.810183,0.891217,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.773216,0.775991,0.785775,0.775767,0.863120,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.779996,0.781983,0.788564,0.781593,0.867920,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.846650,0.847374,0.849606,0.846935,0.916304,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.829822,0.831432,0.836297,0.830436,0.908750,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.809160,0.810010,0.813251,0.809809,0.892261,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.773682,0.776847,0.787983,0.776678,0.863382,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.776250,0.778805,0.788110,0.778655,0.869366,"TfidfVectorizer(analyzer='char', ngram_range=(..."
